## Importação das bibliotecas

In [51]:
import basedosdados as bd
import pandas as pd
from datetime import datetime

## Funções auxiliares

In [3]:
def generate_df(query):
    return bd.read_sql(query, billing_project_id="datario-eduardo-amorim")

## Carregando dataframes para as questões 1 a 5

In [7]:
#Carrega a tabela de Chamados abertos na data 01/04/2023 em um dataframe
query_chamados = """
    SELECT * FROM datario.administracao_servicos_publicos.chamado_1746
    WHERE DATE(data_inicio) = '2023-04-01';
"""

df_chamados = generate_df(query_chamados)

Downloading: 100%|██████████| 73/73 [00:00<00:00, 198.46rows/s]


In [8]:
#Carrega a tabela de Bairros em um dataframe 
query_bairros = """
    SELECT * FROM datario.dados_mestres.bairro;
"""

df_bairros = generate_df(query_bairros)

Downloading: 100%|██████████| 164/164 [00:11<00:00, 14.63rows/s]


## Questões 1 a 5

Como o dataframe "df_chamados" já é fruto de uma consulta da tabela de chamados filtrada pela data desejada em todas as questões (01/04/2023), não será necessário aplicar uma filtragem por esta data em cada etapa.

### 1. Quantos chamados foram abertos no dia 01/04/2023?

Tendo em vista que o dataframe "df_chamados" já está filtrado pela data em questão, apenas se faz necessário contarmos quantas linhas ele possui. Deste modo, ao utilizarmos a função len no dataframe, conseguimos a resposta desejada.

In [15]:
len(df_chamados)

73

Resposta: 73 chamados

### 2. Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?


Para conseguirmos saber o tipo de chamado que teve mais chamados aberto no dia em questão, basta realizarmos a contagem de valores agregando a coluna tipo, e após isso selecionarmos o primeiro valor. Assim, filtrando o dataframe "df_chamados" pela coluna "tipo" e utilizando a função "value_counts", agregamos o dataframe pela coluna desejada e fazemos a contagem de linhas referentes a cada tipo. Por fim, a função "head" com o parâmetro "1", nos retorna o maior valor dessa filtragem.

In [23]:
df_chamados['tipo'].value_counts().head(1)

Poluição sonora    24
Name: tipo, dtype: int64

Resposta: O tipo de chamado com mais chamados abertos no dia 01/04/2023 foi Poluição Sonora, com 24 chamados abertos.

### 3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?


Para encontrarmos a resposta dessa questão, precisamos relacionar as tabelas de chamados e bairros, a fim de contarmos quais bairros possuem mais chamados abertos na data em questão. Assim, ao fazermos o "merge" dos dataframes "df_chamados" e "df_bairros", utilizando a coluna "id_bairro" presente em ambos, conseguimos um dataframe que contem os dados dos chamados abertos no dia 01/04/2023 com as informações referentes ao bairro do chamado. Logo, basta filtrar da mesma maneira mostrada na questão anterior, porém mudando o tipo para "nome", que é a coluna referente ao nome do bairro, e colocar o parâmentro da função "head" para "3", pegando assim os 3 bairros com mais chamados abertos no dia em questão.  

In [28]:
df_chamados_bairro = df_chamados.merge(df_bairros, on="id_bairro")

In [29]:
df_chamados_bairro['nome'].value_counts().head(3)

Engenho de Dentro    8
Leblon               6
Campo Grande         6
Name: nome, dtype: int64

Respostas: Os 3 bairros com mais chamados abertos na data em questão foram: Engenho de Dentro, com 8 chamados; Leblon, com 6 chamados; e Campo Grande, também com 6 chamados.

### 4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

Para a resposta em questão, utilizaremos a mesma relação criada na resposta anterior, porém buscamos o nome da subprefeitura com mais chamados abertos no dia em questão, não os 3 bairros com mais chamados abertos nesse dia. Assim, basta mudarmos a filtragem da coluna "nome" para a coluna "subprefeitura", e mudarmos o parâmetro da função "head" para "1", conseguindo assim a subprefeitura com mais chamados abertos nesse dia.

In [32]:
df_chamados_bairro['subprefeitura'].value_counts().head(1)

Zona Norte    25
Name: subprefeitura, dtype: int64

Resposta: A subprefeitura com mais chamados abertos nesse dia foi a Zona Norte, com 25 chamados.

###  5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

Para essa questão, é necessário apenas contar quais registros de chamados no dataframe "df_chamados" não possuem valor na coluna "id_bairro", visto que isso impossibilita-os de terem um bairro associado. Assim, precisamos apenas utilizar a função "loc", filtrando a coluna "id_bairro" com a função "isna", que retorna os registros com valor nulo na coluna em questão. A partir disto, podemos inferir o motivo do chamado não ter bairro associado analisando as suas colunas. 

In [40]:
df_chamados_sem_bairro = df_chamados.loc[df_chamados['id_bairro'].isna()]

In [41]:
df_chamados_sem_bairro

,id_chamado,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,id_unidade_organizacional_mae,...,tempo_prazo,prazo_unidade,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao,geometry
0,18516246,2023-04-01 00:55:38,2023-04-01 00:55:38,None,None,None,<NA>,1706,TR/SUBOP/CFT - Coordenadoria de Fiscalização e...,SMTR - Secretaria Municipal de Transportes,...,<NA>,D,F,No prazo,Encerrado,Atendido parcialmente,None,0,2023-04-01,None


In [43]:
df_chamados_sem_bairro[['id_chamado', 'id_bairro', 'tipo', 'subtipo']]

,id_chamado,id_bairro,tipo,subtipo
0,18516246,None,Ônibus,Verificação de ar condicionado inoperante no ô...


Resposta: Existe 1 chamado que não possui id_bairro (id_chamado 18516246), logo, não possui bairro associado. Isso se deve pelo fato de ser um chamado do tipo "Ônibus", sendo assim, não possui necessariamente uma localização fixa para o chamado.


## Carregando dataframes para as questões de 6 a 10

In [45]:
#Carrega a tabela de Chamados abertos com o subtipo "Perturbação do sossego" em um dataframe
query_chamados = """
    SELECT * FROM datario.administracao_servicos_publicos.chamado_1746
    WHERE subtipo = 'Perturbação do sossego';
"""

df_chamados = generate_df(query_chamados)

Downloading: 100%|██████████| 90372/90372 [00:34<00:00, 2588.62rows/s]


In [46]:
#Carrega a tabela de Eventos em um dataframe
query_eventos = """
    SELECT * FROM datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos;
"""

df_eventos = generate_df(query_eventos)

Downloading: 100%|██████████| 4/4 [00:00<00:00, 11.47rows/s]


## Questões 6 a 10

Como o dataframe "df_chamados" já é fruto de uma consulta da tabela de chamados filtrada pelo subtipo desejado em todas as questões (Perturbação do sossego), não será necessário aplicar uma filtragem por este subtipo em cada etapa.

###  6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

Para analisarmos quantos chamados foram feitos com o subtipo desejado no intervalo de datas pedido, precisamos apenas filtrar o "df_chamados" pelo intervalo de datas escolhido. Assim, criamos duas variáveis para armazenar a data inicial e a data final do intervalo e filtramos o "df_chamados" pelo intervalo utilizando a função "loc". O filtro foi feito encontrando cada registro do dataframe que possuia a coluna 'data_inicio' (pegando apenas a data, e não o horário) maior ou igual à variável 'data_inicial' e menor ou igual à variável 'data_final'.

Por fim, foi criado um dataframe chamado "df_chamados_filtro_data" para armazenar esta filtragem por data, visto que será utilizada novamente posteriomente, e contamos os seus registros para obtermos a quantidade de chamados pedida.

In [52]:
data_inicial = datetime(2022, 1, 1).date()
data_final = datetime(2023, 12, 31).date()

In [54]:
df_chamados_filtro_data = df_chamados.loc[(df_chamados['data_inicio'].dt.date >= data_inicial) & (df_chamados['data_inicio'].dt.date <= data_final)]

In [56]:
len(df_chamados_filtro_data)

42408

Resposta: 42408 chamados.

###  7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

Para encontrarmos a resposta dessa questão, precisamos relacionar as tabelas de chamados e eventos, a fim de contarmos quantos chamados foram abertos durante a duração dos eventos em questão. Assim, precisamos fazer o 'merge' dos dataframes 'df_chamados' e 'df_eventos', porém, como queremos relacionar uma coluna do dataframe 'df_chamados' ('data_inicio'), com mais de uma coluna do dataframe 'df_eventos' (um intervalo proposto pelas colunas 'data_inicial' e 'data_final'), não conseguirmos fazer um 'merge' diretamente por essas colunas. Logo, precisamos fazer um produto cartesiano entre os dois dataframes, utilizando o tipo de 'merge' chamado 'cross', retornando assim todas as combinações de linhas entre ambos dataframes. Por fim, basta filtrar o resultado da operção pelas datas desejadas, utilizando o mesmo método da questão anterior.

In [66]:
df_chamados_cross = df_chamados.merge(df_eventos, how='cross')

df_chamados_evento = df_chamados_cross.loc[(df_chamados_cross['data_inicio'].dt.date >= df_chamados_cross['data_inicial']) & (df_chamados_cross['data_inicio'].dt.date <= df_chamados_cross['data_final'])]

In [67]:
df_chamados_evento[['id_chamado', 'data_inicio', 'evento']]

,id_chamado,data_inicio,evento
160727,18079637,2023-01-01 04:51:20,Reveillon
161111,18079649,2023-01-01 06:11:34,Reveillon
161115,18079733,2023-01-01 10:14:50,Reveillon
161119,18079648,2023-01-01 06:10:24,Reveillon
161123,18080330,2023-01-01 21:43:40,Reveillon
...,...,...,...
348399,18078185,2022-12-30 20:47:16,Reveillon
349215,18079523,2022-12-31 20:52:48,Reveillon
349219,18077935,2022-12-30 18:08:52,Reveillon
349223,18079445,2022-12-31 19:02:01,Reveillon


In [68]:
len(df_chamados_evento)

1212

Resposta: 1212 chamados.